# S3W11D1: Agent 的心脏——Function Calling 原理

## 🎯 今日目标

1. **核心原理 (First Principles)**: 彻底祛魅 "Agent"，理解 Function Calling 本质上是 **LLM 预测 JSON 参数** + **外部 Python 解释器执行** 的过程。
2. **代码实战 (Hand-On)**: 不依赖 LangChain，直接使用 SiliconFlow API 实现一个最小化的 **"Loop" (LLM 决策 -> 代码执行 -> 结果回填)**。
3. **算法突破 (Algorithm)**: 攻克二叉树重构难题 (LC 105)，掌握递归中的索引切分技巧。
4. **工程规范 (Refactor)**: 按照工业级标准，开始将工具函数封装至 `src/llm/tools.py`。


## 1 理论知识讲解 (First Principles)

### 1.1 核心概念：LLM 并不“思考”，它只是“预测”

很多人误以为 Agent 是 LLM 自己去上网、自己去按计算器。**错**。
LLM 是一个被关在“小黑屋”里的天才作家（Chef）。他唯一的输出就是**文字**。

* **场景**: 用户问“杭州今天天气怎么样？”
* **问题**: LLM 的训练数据截止到 2023 年，它不知道今天的天气。
* **解决方案 (Function Calling)**:
    1. **定义工具**: 我们告诉 LLM：“我有一个工具叫 `get_weather(city_name)`，如果你需要查天气，就按照特定格式（JSON）写给我看。”
    2. **LLM 决策**: LLM 读到用户的这一问，结合上下文，决定不直接回答，而是生成一段 JSON：`{"tool": "get_weather", "args": {"city_name": "Hangzhou"}}`。
    3. **暂停与执行 (Pause & Act)**: 我们的 Python 程序（Runtime）捕获到这段 JSON，程序**暂停** LLM 的生成，在本地运行 Python 函数 `get_weather("Hangzhou")`，得到结果 `"25°C, Sunny"`。
    4. **回填与生成 (Resume)**: 程序把 `"25°C, Sunny"` 作为一条新的“观察消息 (Observation)”喂回给 LLM。LLM 看到结果后，终于生成最终回答：“杭州今天天气晴朗，气温 25 度。”



### 1.2 形象比喻：主厨与帮厨

* **User**: 顾客点餐（提问）。
* **LLM (Agent)**: 主厨。他动口不动手，他写“菜单”（生成 JSON）。
* **Python Runtime**: 帮厨。他看不懂顾客的意图，但他会切菜（执行代码）。
* **Context Window**: 厨房传菜口。主厨的菜单和帮厨切好的菜，都在这里传递。

## 2. 代码实现 (Hand-Written Agent Loop)

我们将使用 **SiliconFlow API** (兼容 OpenAI SDK) 来模拟这个过程。

**前置准备**: 确保你的 `.env` 中有 `SILICONFLOW_API_KEY`。


In [6]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv

# 0. 环境配置
load_dotenv() 

# Debug: 打印一下看看是否读取成功 (正式运行时可注释掉)
# print(f"Key loaded: {os.getenv('LLM_API_KEY') is not None}")

client = OpenAI(
    # 👇 修改这里：使用 .env 中实际定义的变量名
    api_key=os.getenv("DEEPSEEK_API_KEY"), 
    # 👇 修改这里：建议也从环境变量读取 Base URL
    base_url=os.getenv("DEEPSEEK_BASE_URL") 
)

# 1. 定义真实的 Python 函数 (帮厨的工具)
def get_current_weather(location, unit="celsius"):
    """模拟查询天气的函数"""
    print(f"⚡ [System]: 正在调用外部工具查询 {location} 的天气...")
    if "hangzhou" in location.lower() or "杭州" in location:
        return json.dumps({"location": "Hangzhou", "temperature": "22", "unit": unit, "condition": "Cloudy"})
    elif "beijing" in location.lower() or "北京" in location:
        return json.dumps({"location": "Beijing", "temperature": "15", "unit": unit, "condition": "Sunny"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

# 2. 定义工具描述 (给主厨看的说明书 - JSON Schema)
# 这是 OpenAI 标准格式，告诉 LLM 这个工具有什么参数
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "获取指定城市的当前天气情况",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市名称，如 Beijing, Hangzhou",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

# 3. 第一轮对话：用户提问 -> LLM 决定调用工具
messages = [{"role": "user", "content": "杭州今天天气怎么样？"}]

print("Reflecting (LLM 正在思考)...")
response = client.chat.completions.create(
    model="deepseek-chat", # 确保使用支持 Function Calling 的模型
    messages=messages,
    tools=tools,
    tool_choice="auto",  # 让模型自己决定是否用工具
)

response_message = response.choices[0].message
tool_calls = response_message.tool_calls

# 4. 检查 LLM 是否想调用工具
if tool_calls:
    print(f"🤖 [LLM]: 我需要调用工具: {tool_calls[0].function.name}")
    print(f"📝 [Args]: {tool_calls[0].function.arguments}")

    # Step 4.1: 手动执行函数 (Mapping)
    available_functions = {
        "get_current_weather": get_current_weather,
    }
    
    # 把 LLM 的回复加入历史记录 (很重要！否则 LLM 会忘记它刚才想调工具)
    messages.append(response_message)

    # Step 4.2: 真正执行代码
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        
        # 执行 Python 函数
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        
        # Step 4.3: 把执行结果作为 'tool' 类型的消息塞回对话历史
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )

    # 5. 第二轮对话：LLM 看到结果 -> 生成最终回答
    print("Reflecting with Data (LLM 拿到数据再次思考)...")
    final_response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
    )
    print("-----------------------------------")
    print(f"💡 [Final Answer]: {final_response.choices[0].message.content}")
else:
    print("LLM 决定不调用工具，直接回答。")

Reflecting (LLM 正在思考)...
🤖 [LLM]: 我需要调用工具: get_current_weather
📝 [Args]: {"location": "杭州"}
⚡ [System]: 正在调用外部工具查询 杭州 的天气...
Reflecting with Data (LLM 拿到数据再次思考)...
-----------------------------------
💡 [Final Answer]: 根据查询结果，**杭州今天**的天气情况如下：

- **温度**：22°C（体感舒适）
- **天气状况**：多云 ☁️

今天杭州天气比较舒适，温度适中，多云天气适合外出活动。建议您根据个人体感适当穿衣，如果长时间在户外，可以带件薄外套备用。

需要我为您提供未来几天的天气预报吗？


## 3. 深度理论 (Deep Dive)

### 为什么 LLM 能生成 JSON？

这不是巧合。支持 Function Calling 的模型（如 GPT-4, DeepSeek-V2.5, Qwen-2.5-72B）在预训练或微调阶段，被投喂了大量的 **Code 和 API 调用数据**。
它们学会了一种模式：当遇到 `<tool_start>` 这样的特殊 Token 时，不再生成自然语言，而是严格遵循 JSON 语法生成结构化数据。

### ReAct 模式 (Reasoning + Acting)

我们在上面的代码中其实无意间实现了一个简化版的 **ReAct** 循环：

1. **Thought**: LLM 思考用户意图。
2. **Action**: LLM 生成工具调用请求。
3. **Observation**: Python 执行代码，返回结果。
4. **Response**: LLM 根据 Observation 生成回答。

## 4. 模拟面试 (Mock Interview)

**面试官**：请问 OpenAI 的 Function Calling 流程中，模型是真的执行了代码吗？
**黄同学**：
不是的。这是一个常见的误区。

1. **模型侧**：模型本质上只是一个文本生成器。Function Calling 只是让模型根据 System Prompt 中定义的 Schema，**预测**出一个符合 JSON 格式的字符串（包含函数名和参数）。
2. **客户端侧**：真正的代码执行权在我们的 Python 脚本（Client）手中。我们解析模型生成的 JSON，在本地运行函数，然后将运行结果（Observation）以 Message 的形式再次喂给模型。
3. **总结**：模型负责“规划”，程序负责“执行”。这是一个**Human-in-the-loop**（或者说 Program-in-the-loop）的过程。

## 5. 工程封装 (Engineering)

今天我们只是在 Notebook 里跑通了流程。为了后面的项目，我们需要开始规划 `src/` 的结构。

请在 `src/llm/` 下创建或更新以下文件结构：

1. **`src/llm/tools.py`**: 专门存放工具函数（比如今天的 `get_current_weather`，未来的 `search_vector_db`）。
2. **`src/llm/schemas.py`**: 专门存放工具的 JSON 定义（Keep clean）。

**Action Item**: 将今天的 `get_current_weather` 函数移动到 `src/llm/tools.py` 中，并加上规范的 docstring。


## 6. 今日 LeetCode 练习

### LC 105. Construct Binary Tree from Preorder and Inorder Traversal

* **难度**: Medium (经典中的经典)
* **核心思想**: 递归 + 分治。
* **第一性原理**:
* **前序遍历 (Preorder)**: `[根 | 左子树 | 右子树]`。特点：**第一个元素一定是根节点**。
* **中序遍历 (Inorder)**: `[左子树 | 根 | 右子树]`。特点：**根节点把左右子树分开**。


* **解题步骤**:
1. 从 Preorder 拿出第一个数 -> `root`。
2. 在 Inorder 中找到这个 `root` 的位置 `idx`。
3. Inorder 中，`idx` 左边就是左子树的内容，右边就是右子树的内容。
4. 根据 Inorder 左子树的长度，去 Preorder 里截取对应长度的片段。
5. 递归构建。



```python
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right

class Solution:
    def buildTree(self, preorder: List[int], inorder: List[int]) -> Optional[TreeNode]:
        # Base Case: 如果列表为空，返回 None
        if not preorder or not inorder:
            return None
        
        # 1. 根节点是 preorder 的第一个元素
        root_val = preorder[0]
        root = TreeNode(root_val)
        
        # 2. 在 inorder 中找到根节点的位置（切分点）
        # 优化点：可以用 Hashmap 预存 inorder 的索引，避免每次 .index() 造成 O(N^2)
        mid_idx = inorder.index(root_val)
        
        # 3. 切分数据
        # 左子树的 inorder: inorder[0 : mid_idx]
        # 右子树的 inorder: inorder[mid_idx+1 : ]
        
        # 左子树的 size: mid_idx
        # 左子树的 preorder: preorder[1 : 1+mid_idx] (跳过第一个根，取同样长度)
        # 右子树的 preorder: preorder[1+mid_idx : ]
        
        # 4. 递归构建
        root.left = self.buildTree(preorder[1 : 1 + mid_idx], inorder[:mid_idx])
        root.right = self.buildTree(preorder[1 + mid_idx :], inorder[mid_idx+1:])
        
        return root

```

* **复杂度**:
* 如果不优化 `index()` 查找，是 。
* 如果用 Map 存储 `val -> index`，则是 。





## 7. 今日任务总结

* [ ] **Code**: 在 Notebook 中跑通了 SiliconFlow 的 Function Calling 示例，成功让 LLM "查"到了天气。
* [ ] **Concept**: 理解了 Agent = LLM (Planner) + Tools (Executor) + Runtime Loop。
* [ ] **Algorithm**: 攻克了 LC 105，掌握了通过遍历序列还原二叉树的技巧。
* [ ] **Engineering**: 初步规划了 `src/llm/tools.py`。





### 💡 导师点评

黄同学，今天的代码量不大，但**含金量极高**。你刚刚跨越了“生成式 AI”到“代理式 AI”的门槛。
当你看到 Terminal 里打印出 `🤖 [LLM]: 我需要调用工具...` 时，那种通过自然语言控制程序逻辑的感觉，是不是很美妙？

明天，我们要把这个能力应用到我们的 RAG 系统上，让它**只在需要的时候查向量库，而不是胡乱检索**。好好休息！